In [ ]:
!pip install spacy -U
!pip install spacy-transformers
!pip install pytesseract -U
!apt install tesseract-ocr
!apt install libtesseract-dev

In [ ]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.7.3                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-6.1.58+-x86_64-with-glibc2.35
Python version   3.10.12                       
Pipelines        en_core_web_sm (3.7.1)        



In [ ]:
import spacy
import pandas as pd
import cv2 as cv
import pytesseract
import matplotlib.pyplot as plt
import textwrap

In [ ]:
nlp_ner = spacy.load("/content/drive/MyDrive/4th_Year/DSC4996/Spacy/model-best")
df = pd.read_excel('/content/drive/MyDrive/4th_Year/DSC4996/Preprocessed_data.xlsx')

In [ ]:
def pytes(image):
  result = pytesseract.image_to_string(image)
  result = ' '.join(result.split())
  result = result.replace('  ', ' ').replace('   ', ' ').replace('    ', ' ')
  return result

In [ ]:
img = cv.imread(f'/content/drive/MyDrive/4th_Year/DSC4996/Claim_forms/23.png')
plt.imshow(img)
plt.axis('off')

In [ ]:
input = pytes(img)
doc = nlp_ner(input)
spacy.displacy.render(doc, style="ent", jupyter=True)

In [ ]:
Policy_no = None
Request_date = None
Doctor_name = None
Hospital_name = None
Amount = None
Types = {}
Types_in_claim_form = []

In [ ]:
policy_number_temp = []

for entity in doc.ents:
  if entity.label_ == 'POLICY_NO':
    policy_number_temp.append(entity.text.replace('NGTO', 'NGT0').replace('L14', 'LI4'))
    Policy_no = '-'.join(policy_number_temp)
  elif entity.label_ == 'DATE':
    Request_date = entity.text
  elif entity.label_ == 'DOCTOR':
    Doctor_name = entity.text
  elif entity.label_ == 'HOSPITAL':
    Hospital_name = entity.text
  elif entity.label_ == 'AMOUNT':
    Amount = int(entity.text)
  else:
    if entity.label_ not in Types:
      Types[f'{entity.label_}'] = []
    if entity.label_ not in Types_in_claim_form:
      Types_in_claim_form.append(entity.label_)
    Types[f'{entity.label_}'].append(entity.text)

In [ ]:
temp_df = df.iloc[list(df['PolicyNo'] == Policy_no)]
Types_in_DB = temp_df.groupby('TYPE').size().index.tolist()

registered_types = []
unregistered_types = []
for typ in Types_in_claim_form:
  if typ in Types_in_DB:
    registered_types.append(typ)
  else:
    unregistered_types.append(typ)
    continue

In [ ]:
print('\n* * * * * * * * * * * * * * * * * * * * SUMMARY * * * * * * * * * * * * * * * * * * * *\n\n')
print(f'(1) POLICY NUMBER :- {Policy_no}\n')
line_1 = f'(1) POLICY NUMBER :- {Policy_no}'
print(f'(2) ACCIDENT DATE :- {Request_date}\n')
line_2 = f'(2) ACCIDENT DATE  :- {Request_date}'
print(f'(3) DOCTOR NAME   :- {Doctor_name}\n')
line_3 = f'(3) DOCTOR NAME   :- {Doctor_name}'
print(f'(4) HOSPITAL NAME :- {Hospital_name}\n')
line_4 = f'(4) HOSPITAL NAME  :- {Hospital_name}'
print(f'(5) AMOUNT        :- Rs.{Amount}\n')
line_5 = f'(5) AMOUNT         :- Rs.{Amount}'
print('(6) CLAIM DETAILS :- ')
line_6 = '(6) CLAIM DETAILS   :- '

claiming_coverages = []
for key, val in Types.items():
  print('\t\t    ', key, ':-', ', '.join(val))
  temp = f"{key} :- {', '.join(val)}"
  claiming_coverages.append(temp)


exp_date = temp_df.groupby('EXPIRYDATE').size().index[0]
exp_date = pd.to_datetime(exp_date, dayfirst = True)
Request_date = pd.to_datetime(Request_date, dayfirst = True)

if Request_date > exp_date:
    print(f'\n(7) CLAIM STATUS :- REJECTED\n')
    line_7 = '(7) CLAIM STATUS :- REJECTED'
    print('(8) REJECTION DETAILS :- All your life insurance coverages have expired.\n')
    line_8 = '(8) REJECTION DETAILS :- All your life insurance coverages have expired.'
else:
    if len(registered_types) != 0:
      total_suminsured = 0
      for typ in registered_types:
        temp_amount = sum(temp_df.iloc[list(temp_df['TYPE'] == typ)]['SUMINSURED'].tolist())
        total_suminsured += temp_amount
      if len(unregistered_types) != 0:
        if total_suminsured <= Amount:
          print('\n(7) CLAIM STATUS :- ACCEPTED\n')
          line_7 = '(7) CLAIM STATUS :- ACCEPTED'
          print(f'(8) PAYMENT NOTE :- Dear valuable {Policy_no} policy holder, you are not registered for {", ".join(unregistered_types)} coverages. Therefore we can only payfor your {", ".join(registered_types)} coverages. We cannot pay Rs.{Amount}. We are obligated to pay Rs.{int(total_suminsured)} only.\n')
          line_8 = f'(8) PAYMENT NOTE :- Dear valuable {Policy_no} policy holder, you are not registered for {", ".join(unregistered_types)} coverages. Therefore we can only payfor your {", ".join(registered_types)} coverages. We cannot pay Rs.{Amount}. We are obligated to pay Rs.{int(total_suminsured)} only.'
          print(f'(9) ACCEPTED PAYING AMOUNT :- Rs.{int(total_suminsured)}\n')
          line_9 = f'(9) ACCEPTED PAYING AMOUNT :- Rs.{int(total_suminsured)}'
        else:
          print('\n(7) CLAIM STATUS :- ACCEPTED\n')
          line_7 = '(7) CLAIM STATUS :- ACCEPTED'
          print(f'(8) PAYMENT NOTE :- Dear valuable {Policy_no} policy holder, you are not registered for {", ".join(unregistered_types)} coverages. Therefore we can only pay for your {", ".join(registered_types)} coverages. We are obligated to pay Rs.{int(Amount)} only.\n')
          line_8 = f'(8) PAYMENT NOTE :- Dear valuable {Policy_no} policy holder, you are not registered for {", ".join(unregistered_types)} coverages. Therefore we can only pay for your {", ".join(registered_types)} coverages. We are obligated to pay Rs.{int(Amount)} only.'
          print(f'(9) ACCEPTED PAYING AMOUNT :- Rs.{int(Amount)}\n')
          line_9 = f'(9) ACCEPTED PAYING AMOUNT :- Rs.{int(Amount)}'
      else:
        if total_suminsured <= Amount:
          print('\n(7) CLAIM STATUS :- ACCEPTED\n')
          line_7 = '(7) CLAIM STATUS :- ACCEPTED'
          print(f"(8) PAYMENT NOTE :- Dear valuable {Policy_no} policy holder, you're sum insured coverage limit is reached. Therefor we cannot pay Rs.{Amount}. We are obligated to pay Rs.{int(total_suminsured)} only.\n")
          line_8 = f"(8) PAYMENT NOTE :- Dear valuable {Policy_no} policy holder, you're sum insured coverage limit is reached. Therefor we cannot pay Rs.{Amount}. We are obligated to pay Rs.{int(total_suminsured)} only."
          print(f'(9) ACCEPTED PAYING AMOUNT :- Rs.{int(total_suminsured)}\n')
          line_9 = f'(9) ACCEPTED PAYING AMOUNT :- Rs.{int(total_suminsured)}'
        else:
          print('\n(7) CLAIM STATUS :- ACCEPTED')
          line_7 = '(7) CLAIM STATUS :- ACCEPTED'
          print(f'(8) PAYMENT NOTE :- Dear valuable {Policy_no} policy holder, We are obligated to pay Rs.{int(Amount)}.\n')
          line_8 = f'(8) PAYMENT NOTE :- Dear valuable {Policy_no} policy holder, We are obligated to pay Rs.{int(Amount)}.'
          print(f'(9) ACCEPTED PAYING AMOUNT :- Rs.{Amount}\n')
          line_9 = f'(9) ACCEPTED PAYING AMOUNT :- Rs.{Amount}'
    else:
      print('\n(7) CLAIM STATUS :- REJECTED\n')
      line_7 = '(7) CLAIM STATUS :- REJECTED'
      print(f'(8) REJECTION DETAILS :- Dear valuable {Policy_no} policy holder, you are not registered for all of the coveragesyou have requested.\n')
      line_8 = f'(8) REJECTION DETAILS :- Dear valuable {Policy_no} policy holder, you are not registered for all of the coveragesyou have requested.'
      print(f'(9) UNREGISTERED COVERAGES :- {", ".join(unregistered_types)}\n')
      line_9 = f'(9) UNREGISTERED COVERAGES :- {", ".join(unregistered_types)}'
      print(f'(10)REGISTERED COVERAGES :- {", ".join(Types_in_DB)}')
      line_10 = f'(10)REGISTERED COVERAGES :- {", ".join(Types_in_DB)}'



* * * * * * * * * * * * * * * * * * * * SUMMARY * * * * * * * * * * * * * * * * * * * *


(1) POLICY NUMBER :- JCA01224286

(2) ACCIDENT DATE :- 09-11-2022

(3) DOCTOR NAME   :- DR BANDULA Hospital

(4) HOSPITAL NAME :- DURDANS HOSPITAL

(5) AMOUNT        :- Rs.25000

(6) CLAIM DETAILS :- 
		     CASHLESS HOSPITAL COVER :- medical expenses
		     CRITICAL ILLNESS COVER :- burn treatment, skin grafts, psychological counseling, long-term care costs, specialized clothing and equipment
		     ADDITIONAL CRITICAL ILLNESS COVER :- multiple surgeries
		     PERSONAL ACCIDENT COVER :- potential home modifications

(7) CLAIM STATUS :- REJECTED

(8) REJECTION DETAILS :- Dear valuable JCA01224286 policy holder, you are not registered for all of the coveragesyou have requested.

(9) UNREGISTERED COVERAGES :- CASHLESS HOSPITAL COVER, CRITICAL ILLNESS COVER, ADDITIONAL CRITICAL ILLNESS COVER, PERSONAL ACCIDENT COVER

(10)REGISTERED COVERAGES :- ADDITIONAL ACCIDENT COVER, ADDITIONAL HOSPITAL COVER, 

In [ ]:
img_out1 = cv.imread('/content/drive/MyDrive/4th_Year/DSC4996/claim details.png')
img_out1 = cv.cvtColor(img_out1, cv.COLOR_BGR2RGB)
img_out1.shape

(2000, 1414, 3)

In [ ]:
def wrapText(width, text, image, x, y, gap):
  wrapper = textwrap.TextWrapper(width=width)
  word_list = wrapper.wrap(text=text)

  for words in word_list:
    img_out = cv.putText(img = image, text = words, org = (x, y), fontFace = fontface, fontScale = fontscale, color = clr, thickness = thick)

    if words == word_list[-1]:
      y += gap
    else:
      y += 38
  return image, y

In [ ]:
x = 130
y = 650
fontface = cv.FONT_HERSHEY_SIMPLEX
fontscale = 1
clr = (0, 0, 0)
thick = 2
gap = 80

img_out = img_out1.copy()

#line 1
img_out = cv.putText(img = img_out, text = line_1, org = (x, y), fontFace = fontface, fontScale = fontscale, color = clr, thickness = thick)
y += gap

#line 2
img_out = cv.putText(img = img_out, text = line_2, org = (x, y), fontFace = fontface, fontScale = fontscale, color = clr, thickness = thick)
y += gap

#line 3
img_out = cv.putText(img = img_out, text = line_3, org = (x, y), fontFace = fontface, fontScale = fontscale, color = clr, thickness = thick)
y += gap

#line 4
img_out = cv.putText(img = img_out, text = line_4, org = (x, y), fontFace = fontface, fontScale = fontscale, color = clr, thickness = thick)
y += gap

#line 5
img_out = cv.putText(img = img_out, text = line_5, org = (x, y), fontFace = fontface, fontScale = fontscale, color = clr, thickness = thick)
y += gap

#line 6
img_out = cv.putText(img = img_out, text = line_6, org = (x, y), fontFace = fontface, fontScale = fontscale, color = clr, thickness = thick)
y += (gap-30)

#line 6.1
x = 250
for tmp in claiming_coverages:
  img_out, y = wrapText(57, tmp, img_out, x, y, 55)

  if tmp == claiming_coverages[-1]:
    y += 25

#line 7
x = 130
img_out = cv.putText(img = img_out, text = line_7, org = (x, y), fontFace = fontface, fontScale = fontscale, color = clr, thickness = thick)
y += gap

#line 8
img_out, y = wrapText(69, line_8, img_out, x, y, gap)

#line 9
img_out, y = wrapText(65, line_9, img_out, x, y, gap)

#line 10
img_out, _ = wrapText(65, line_10, img_out, x, y, gap)

In [ ]:
fig = plt.figure(figsize=(10, 15))
plt.imshow(img_out)

In [ ]:
import spacy
import pandas as pd
import cv2 as cv
import pytesseract
import matplotlib.pyplot as plt


def pytes(image):
  result = pytesseract.image_to_string(image)
  result = ' '.join(result.split())
  result = result.replace('  ', ' ').replace('   ', ' ').replace('    ', ' ')
  return result


nlp_ner = spacy.load("/content/drive/MyDrive/4th_Year/DSC4996/Spacy/model-best")
df = pd.read_excel('/content/drive/MyDrive/4th_Year/DSC4996/Preprocessed_data.xlsx')


img = cv.imread(f'/content/drive/MyDrive/4th_Year/DSC4996/Claim_forms/44.png')
plt.imshow(img)
plt.axis('off')


input = pytes(img)
doc = nlp_ner(input)
spacy.displacy.render(doc, style="ent", jupyter=True)


Policy_no = None
Request_date = None
Doctor_name = None
Hospital_name = None
Amount = None
Types = {}
Types_in_claim_form = []


policy_number_temp = []
for entity in doc.ents:
  if entity.label_ == 'POLICY_NO':
    policy_number_temp.append(entity.text.replace('NGTO', 'NGT0').replace('L14', 'LI4'))
    Policy_no = '-'.join(policy_number_temp)
  elif entity.label_ == 'DATE':
    Request_date = entity.text
  elif entity.label_ == 'DOCTOR':
    Doctor_name = entity.text
  elif entity.label_ == 'HOSPITAL':
    Hospital_name = entity.text
  elif entity.label_ == 'AMOUNT':
    Amount = int(entity.text)
  else:
    if entity.label_ not in Types:
      Types[f'{entity.label_}'] = []
    if entity.label_ not in Types_in_claim_form:
      Types_in_claim_form.append(entity.label_)
    Types[f'{entity.label_}'].append(entity.text)


temp_df = df.iloc[list(df['PolicyNo'] == Policy_no)]
Types_in_DB = temp_df.groupby('TYPE').size().index.tolist()


registered_types = []
unregistered_types = []
for typ in Types_in_claim_form:
  if typ in Types_in_DB:
    registered_types.append(typ)
  else:
    unregistered_types.append(typ)
    continue


print('\n* * * * * * * * * * * * * * * * * * * * SUMMARY * * * * * * * * * * * * * * * * * * * *\n\n')
print(f'(1)  POLICY NUMBER :- {Policy_no}\n')
print(f'(2)  ACCIDENT DATE :- {Request_date}\n')
print(f'(3)  DOCTOR NAME   :- {Doctor_name}\n')
print(f'(4)  HOSPITAL NAME :- {Hospital_name}\n')
print(f'(5)  AMOUNT        :- Rs.{Amount}\n')
print('(6)  CLAIM DETAILS :- ')
for key, val in Types.items():
  print('\t\t    ', key, ':-', ', '.join(val))


exp_date = temp_df.groupby('EXPIRYDATE').size().index[0]
exp_date = pd.to_datetime(exp_date, dayfirst = True)
Request_date = pd.to_datetime(Request_date, dayfirst = True)

if Request_date > exp_date:
  print('\n(7)  CLAIM STATUS :- REJECTED CLAIM REQUEST\n')
  print(f"(8)  REJECTION DETAILS :- All your life insurance coverages have expired.")
else:
  print('\n(7)  CLAIM STATUS :- ACCEPTED CLAIM REQUEST\n')
  if len(registered_types) != 0:
    total_suminsured = 0
    for typ in registered_types:
      temp_amount = sum(temp_df.iloc[list(temp_df['TYPE'] == typ)]['SUMINSURED'].tolist())
      total_suminsured += temp_amount
    if len(unregistered_types) != 0:
      if total_suminsured <= Amount:
        print(f'''(8)  PAYMENT NOTE :- Dear valuable {Policy_no} policy holder, you are not registered for {", ".join(unregistered_types)} coverages.
                     Therefore we can only pay for your {", ".join(registered_types)} coverages. We cannot pay Rs.{Amount}. We are obligated to pay Rs.{int(total_suminsured)} only.''')
      else:
        print(f'''(8)  PAYMENT NOTE :- Dear valuable {Policy_no} policy holder, you are not registered for {", ".join(unregistered_types)} coverages. Therefore we can only pay for your {", ".join(registered_types)} coverages.
                     We are obligated to pay Rs.{(Amount)} only.''')
    else:
      if total_suminsured <= Amount:
        print(f'''(8)  PAYMENT NOTE :- Dear valuable {Policy_no} policy holder, you're sum insured coverage limit is reached. Therefor we cannot pay Rs.{Amount}. We are obligated to pay Rs.{int(total_suminsured)} only.''')
      else:
        print(f'''(8)  PAYMENT NOTE :- Dear valuable {Policy_no} policy holder, We are obligated to pay Rs.{(Amount)} only.''')
  else:
    print('\n(8)  CLAIM STATUS :- REJECTED CLAIM REQUEST\n')
    print(f"(9)  REJECTION DETAILS :- You are not registered for all of the coverages you have requested.\n")
    print(f'(10) UNREGISTERED COVERAGES :- {", ".join(unregistered_types)}\n')
    print(f"(11) REGISTERED COVERAGES :- {', '.join(Types_in_DB)}")

print('\n\n* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *\n')